## DOCHERTY Ronan et HOUNGUEVOU Thomas
## Rapport réseaux de neurones
### Sujet: Génération de synopsis d'anime
Réseau utilisé: **Transformer**  
[Dataset](https://www.kaggle.com/datasets/marlesson/myanimelist-dataset-animes-profiles-reviews/data)

In [18]:
import pandas as pd

# Chargement des données depuis un fichier CSV (ou tout autre format de fichier)
data = pd.read_csv('animes.csv')

data = data[['title', 'synopsis','genre','score']]

# Affichage des premières lignes pour visualiser la structure des données
print(data.head())

0        Following their participation at the Inter-Hig...
1        Music accompanies the path of the human metron...
2        The Abyss—a gaping chasm stretching down into ...
3        "In order for something to be obtained, someth...
4        After helping revive the legendary vampire Kis...
                               ...                        
19306    Cocona is an average middle schooler living wi...
19307    While visiting the National Library, junior-hi...
19308    Years ago, all of the ghosts in a haunted scho...
19309    Inuyasha and company have finally destroyed Na...
19310    The year is Universal Century 0093. Char Aznab...
Name: synopsis, Length: 19311, dtype: object


In [11]:
from sklearn.model_selection import train_test_split

# Séparation des données en ensembles d'entraînement et de test (par exemple, 80% pour l'entraînement et 20% pour le test)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Affichage de la taille des ensembles d'entraînement et de test
print("Taille de l'ensemble d'entraînement :", len(train_data))
print("Taille de l'ensemble de test :", len(test_data))

Taille de l'ensemble d'entraînement : 15448
Taille de l'ensemble de test : 3863


In [ ]:
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers

synopsis_list = train_data['synopsis'].astype(str).tolist()  # Assurez-vous que les données sont bien de type str

# Initialisation du tokenizer
tokenizer = Tokenizer(models.BPE())

# Configuration du trainer
trainer = trainers.BpeTrainer(vocab_size=10000, special_tokens=['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]'])

# Entraînement du tokenizer sur les synopsis nettoyés
tokenizer.train_from_iterator(synopsis_list, trainer=trainer)

In [24]:
import torch
import torch.nn as nn

class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embedding_size, num_layers, num_heads, hidden_size):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.transformer = nn.Transformer(
            d_model=embedding_size,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=hidden_size
        )
        self.linear = nn.Linear(embedding_size, vocab_size)

    def forward(self, src):
        embedded = self.embedding(src)
        output = self.transformer(embedded, embedded)
        output = self.linear(output)
        return output